# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests
import json

# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Windspeed,Country,Date
0,0,papatowai,-46.5619,169.4708,7.48,69,18,6.01,NZ,1724960019
1,1,adamstown,-25.0660,-130.1015,19.15,70,58,6.04,PN,1724960020
2,2,changzhi,35.2089,111.7386,22.19,92,0,1.09,CN,1724960021
3,3,port-aux-francais,-49.3500,70.2167,3.76,78,68,16.17,TF,1724960023
4,4,grytviken,-54.2811,-36.5092,0.41,96,100,1.01,GS,1724960025


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [4]:
%%capture --no-display

# Configure the map plot
map_plot_1 = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 800,
    frame_height = 800,
    size = "Humidity",
    scale = 0.75,
    color = "City"
)

# Display the map
map_plot_1

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [5]:
# Narrow down cities that fit criteria and drop any results with null values
ideal_weather = city_data_df[(city_data_df["Humidity"] < 75) & (city_data_df["Windspeed"] < 4) & (city_data_df["Cloudiness"] == 0) &
                            (city_data_df["Max Temp"] < 27) & (city_data_df["Max Temp"] > 20)]

# Drop any rows with null values
ideal_weather = ideal_weather.dropna()

# Display sample data
ideal_weather

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Windspeed,Country,Date
57,57,kolonia,50.9333,6.9500,24.97,70,0,1.54,DE,1724959832
261,261,alba adriatica,42.8318,13.9259,26.62,69,0,1.67,IT,1724960316
307,307,lewiston orchards,46.3804,-116.9754,26.39,27,0,1.14,US,1724960369
338,338,vernon,50.2581,-119.2691,22.64,49,0,1.03,CA,1724960405
352,352,saint-henri,46.6921,-71.0671,20.99,55,0,1.03,CA,1724960421
371,371,spring creek,40.7266,-115.5859,23.55,12,0,2.75,US,1724960443
433,433,ifrane,33.3333,-5.2500,21.81,50,0,1.90,MA,1724960514
504,504,maun,-19.9833,23.4167,24.98,12,0,3.60,BW,1724960595
518,518,beyneu,45.3167,55.2000,25.76,30,0,2.40,KZ,1724960610
561,561,orosei,40.3798,9.6929,25.22,69,0,1.40,IT,1724960661


### Step 3: Create a new DataFrame called `hotel_df`.

In [6]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = ideal_weather.copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Windspeed,Country,Date,Hotel Name
57,57,kolonia,50.9333,6.9500,24.97,70,0,1.54,DE,1724959832,
261,261,alba adriatica,42.8318,13.9259,26.62,69,0,1.67,IT,1724960316,
307,307,lewiston orchards,46.3804,-116.9754,26.39,27,0,1.14,US,1724960369,
338,338,vernon,50.2581,-119.2691,22.64,49,0,1.03,CA,1724960405,
352,352,saint-henri,46.6921,-71.0671,20.99,55,0,1.03,CA,1724960421,
371,371,spring creek,40.7266,-115.5859,23.55,12,0,2.75,US,1724960443,
433,433,ifrane,33.3333,-5.2500,21.81,50,0,1.90,MA,1724960514,
504,504,maun,-19.9833,23.4167,24.98,12,0,3.60,BW,1724960595,
518,518,beyneu,45.3167,55.2000,25.76,30,0,2.40,KZ,1724960610,
561,561,orosei,40.3798,9.6929,25.22,69,0,1.40,IT,1724960661,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [8]:
# Set parameters to search for a hotel
radius = 10000
params = {
    "categories":"accommodation.hotel",
    "apiKey": geoapify_key,
    "limit":20
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    latitude = row["Lat"]
    longitude = row["Lng"]
    
    # Add the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"
    params["limit"] = 10
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make and API request using the params dictionary
    name_address = requests.get(base_url, params=params)

    # Convert the API response to JSON format
    name_address = name_address.json()

    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"

    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
kolonia - nearest hotel: Wasserturm Hotel Cologne
alba adriatica - nearest hotel: Hotel B&B Villa Cesare
lewiston orchards - nearest hotel: Hampton Inn Lewiston
vernon - nearest hotel: Okanagan Royal Park Inn
saint-henri - nearest hotel: No hotel found
spring creek - nearest hotel: No hotel found
ifrane - nearest hotel: la perle d'azrou
maun - nearest hotel: Center Lodge Conference Center
beyneu - nearest hotel: Нұр
orosei - nearest hotel: Hotel Mannois


,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Windspeed,Country,Date,Hotel Name
57,57,kolonia,50.9333,6.9500,24.97,70,0,1.54,DE,1724959832,Wasserturm Hotel Cologne
261,261,alba adriatica,42.8318,13.9259,26.62,69,0,1.67,IT,1724960316,Hotel B&B Villa Cesare
307,307,lewiston orchards,46.3804,-116.9754,26.39,27,0,1.14,US,1724960369,Hampton Inn Lewiston
338,338,vernon,50.2581,-119.2691,22.64,49,0,1.03,CA,1724960405,Okanagan Royal Park Inn
352,352,saint-henri,46.6921,-71.0671,20.99,55,0,1.03,CA,1724960421,No hotel found
371,371,spring creek,40.7266,-115.5859,23.55,12,0,2.75,US,1724960443,No hotel found
433,433,ifrane,33.3333,-5.2500,21.81,50,0,1.90,MA,1724960514,la perle d'azrou
504,504,maun,-19.9833,23.4167,24.98,12,0,3.60,BW,1724960595,Center Lodge Conference Center
518,518,beyneu,45.3167,55.2000,25.76,30,0,2.40,KZ,1724960610,Нұр
561,561,orosei,40.3798,9.6929,25.22,69,0,1.40,IT,1724960661,Hotel Mannois


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [10]:
%%capture --no-display

# Configure the map plot
map_plot_2 = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 800,
    frame_height = 800,
    size = "Humidity",
    scale = 1.75,
    color = "City",
    hover_cols = ["Country", "Hotel Name"]
    
)
# Display the map
map_plot_2

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Country,Hotel Name)